In [3]:
#WORD2VEC model

import logging
import pymysql
import multiprocessing
import gensim
from gensim.models import Word2Vec

from google.colab import drive
drive.mount('/gdrive')

from multiprocessing import cpu_count
cpu_count = cpu_count()
print(f"Количество потоков: {cpu_count}")

import numpy as np

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

# Create the tagged document needed for Doc2Vec
#def create_tagged_document(list_of_list_of_words):
#    for i, list_of_words in enumerate(list_of_list_of_words):
#        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])


connection = get_connection()

EPOCHS_0 = 20 #эпохи первичное обучение
EPOCHS_1 = 20 #эпохи дообучение
COUNT_DB = 10314497 #число строк в обработанной БД
LIMIT_0 = 2000000 #загружаем столько строк для первичного обучения модели
LIMIT_1 = LIMIT_0  #загружаем столько строк для последующего обучения модели

SIZE = 1000 #размерность выходного вектора признаков
ITER = 0 #C какой итерации дообучать (0 - первичное обучение)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
try:
    with connection.cursor() as cursor:
        if ITER == 0:
            
            #загрузка данных для iter_0
            s = "SELECT ttext_stem FROM sentiment_v3 ORDER BY id ASC LIMIT %s"
            cursor.execute(s, (LIMIT_0))
            result = cursor.fetchall()
            result = [doc['ttext_stem'].split('|') for doc in result]
            print(f"Считано {len(result)} строк из базы данных")

            #вычисление конечного id для ITER_0
            s = "SELECT max(id) FROM (SELECT id from sentiment_v3 ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (LIMIT_0))
            end_id = cursor.fetchone()['max(id)']
            print(end_id)

            #вычисление максимального id для всей БД
            s = "SELECT max(id) FROM sentiment_v3"
            cursor.execute(s)
            max_id = cursor.fetchone()['max(id)']

            train_data = result

            #instantiate our DM and DBOW models
            model = Word2Vec(min_count=5, window=5,  size=SIZE, sample=1e-3, negative=5, workers=cpu_count)
            
            #build vocab over all reviews
            model.build_vocab(train_data, update = False)

            #train
            model.train(sentences=train_data, epochs=EPOCHS_0, total_examples=model.corpus_count)
            model.save("/gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_0.word2vec", sep_limit = 100 * 1024 ** 2)
            #model.delete_temporary_training_data()
          
            print(f"Модель model_dm записана на гугл диск")
            COUNT_DB -= LIMIT_0
            print(f"Осталось обработать {COUNT_DB} записей!")
            ITER += 1

        else:
            #вычисление конечного id для ITER_X
            s = "SELECT max(id) FROM (SELECT id from sentiment_v3 ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (LIMIT_0 + LIMIT_1 * (ITER - 1)))
            end_id = cursor.fetchone()['max(id)']
            print(end_id)

            #вычисление максимального id для всей БД
            s = "SELECT max(id) FROM sentiment_v3"
            cursor.execute(s)
            max_id = cursor.fetchone()['max(id)']

        
        # дообучение
        while end_id < max_id:
            
            print(f"Итерация {ITER}:")
            model_old_pach = "/gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_" + str(ITER - 1) + ".word2vec"
            model_pach = "/gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_" + str(ITER) + ".word2vec"


            #загрузка данных для iter_X
            s = "SELECT ttext_stem FROM sentiment_v3 WHERE id > %s ORDER BY id ASC LIMIT %s"
            cursor.execute(s, (end_id, LIMIT_1))
            result = cursor.fetchall()
            result = [doc['ttext_stem'].split('|') for doc in result]
            print(f"Считано {len(result)} строк из базы данных")

            #вычисление конечного id для ITER_X
            s = "SELECT max(id) FROM (SELECT id FROM sentiment_v3 WHERE id > %s ORDER BY id ASC LIMIT %s) AS t"
            cursor.execute(s, (end_id, LIMIT_1))
            end_id = cursor.fetchone()['max(id)']

            train_data = result

            #instantiate our models

            model = Word2Vec.load(model_old_pach) #, mmap='r')  
            
            #build vocab over all reviews
            model.build_vocab(train_data, update = True)
            
            #train
            model.train(sentences=train_data, epochs=EPOCHS_1, total_examples=model.corpus_count)

            #model.delete_temporary_training_data()

            model.save(model_pach, sep_limit = 100 * 1024 ** 2)
            print(f"Модель записана на гугл диск")

            COUNT_DB -= LIMIT_1
            print(f"Осталось обработать {COUNT_DB} записей!")

            ITER += 1
finally:
    connection.close()




Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Количество потоков: 2
Считано 2000000 строк из базы данных


2020-07-23 14:18:20,138 : INFO : collecting all words and their counts
2020-07-23 14:18:20,139 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


411050803860733952


2020-07-23 14:18:20,172 : INFO : PROGRESS: at sentence #10000, processed 93652 words, keeping 20548 word types
2020-07-23 14:18:20,205 : INFO : PROGRESS: at sentence #20000, processed 185437 words, keeping 32984 word types
2020-07-23 14:18:20,235 : INFO : PROGRESS: at sentence #30000, processed 275526 words, keeping 43518 word types
2020-07-23 14:18:20,266 : INFO : PROGRESS: at sentence #40000, processed 366213 words, keeping 53081 word types
2020-07-23 14:18:20,296 : INFO : PROGRESS: at sentence #50000, processed 457117 words, keeping 62122 word types
2020-07-23 14:18:20,326 : INFO : PROGRESS: at sentence #60000, processed 543351 words, keeping 70229 word types
2020-07-23 14:18:20,357 : INFO : PROGRESS: at sentence #70000, processed 622762 words, keeping 76871 word types
2020-07-23 14:18:20,388 : INFO : PROGRESS: at sentence #80000, processed 702311 words, keeping 83357 word types
2020-07-23 14:18:20,419 : INFO : PROGRESS: at sentence #90000, processed 783121 words, keeping 89648 word

Модель model_dm записана на гугл диск
Осталось обработать 8314497 записей!
Итерация 1:
Считано 2000000 строк из базы данных


...

Модель записана на гугл диск
Осталось обработать -3685503 записей!
Итерация 7:
Считано 2000000 строк из базы данных


2020-07-23 17:20:53,760 : INFO : loading Word2Vec object from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec
2020-07-23 17:20:54,114 : INFO : loading wv recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec.wv.* with mmap=None
2020-07-23 17:20:54,115 : INFO : loading vectors from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec.wv.vectors.npy with mmap=None
2020-07-23 17:21:04,276 : INFO : setting ignored attribute vectors_norm to None
2020-07-23 17:21:04,277 : INFO : loading vocabulary recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec.vocabulary.* with mmap=None
2020-07-23 17:21:04,280 : INFO : loading trainables recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec.trainables.* with mmap=None
2020-07-23 17:21:04,282 : INFO : loading syn1neg from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_6.word2vec.trainables.syn1neg.npy with mmap=None

Модель записана на гугл диск
Осталось обработать -5685503 записей!
Итерация 8:
Считано 696346 строк из базы данных


2020-07-23 17:51:11,933 : INFO : loading Word2Vec object from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec
2020-07-23 17:51:12,288 : INFO : loading wv recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec.wv.* with mmap=None
2020-07-23 17:51:12,289 : INFO : loading vectors from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec.wv.vectors.npy with mmap=None
2020-07-23 17:51:16,168 : INFO : setting ignored attribute vectors_norm to None
2020-07-23 17:51:16,170 : INFO : loading vocabulary recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec.vocabulary.* with mmap=None
2020-07-23 17:51:16,173 : INFO : loading trainables recursively from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec.trainables.* with mmap=None
2020-07-23 17:51:16,177 : INFO : loading syn1neg from /gdrive/My Drive/Colab Notebooks/w2v_1000/model1000_iter_7.word2vec.trainables.syn1neg.npy with mmap=None

Модель записана на гугл диск
Осталось обработать -7685503 записей!


In [4]:
'''
#Обновление модели doc2vec
import logging
import pymysql
import multiprocessing
import gensim
from gensim.models import Doc2Vec

from google.colab import drive
drive.mount('/gdrive')

from multiprocessing import cpu_count
cpu_count = cpu_count()
print(f"Количество потоков: {cpu_count}")

import numpy as np

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test1',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection

# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])


connection = get_connection()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
try:
    with connection.cursor() as cursor:

        
  

finally:
    connection.close()
'''

'\n#Обновление модели doc2vec\nimport logging\nimport pymysql\nimport multiprocessing\nimport gensim\nfrom gensim.models import Doc2Vec\n\nfrom google.colab import drive\ndrive.mount(\'/gdrive\')\n\nfrom multiprocessing import cpu_count\ncpu_count = cpu_count()\nprint(f"Количество потоков: {cpu_count}")\n\nimport numpy as np\n\n# Connect to the database\ndef get_connection():\n    connection = pymysql.connect(host=\'database-3.chim8btj05zt.us-east-1.rds.amazonaws.com\',\n                                 user=\'admin\',\n                                 password=\'01035007\',\n                                 db=\'test1\',\n                                 charset=\'utf8mb4\',\n                                 cursorclass=pymysql.cursors.DictCursor)\n    return connection\n\n# Create the tagged document needed for Doc2Vec\ndef create_tagged_document(list_of_list_of_words):\n    for i, list_of_words in enumerate(list_of_list_of_words):\n        yield gensim.models.doc2vec.TaggedDocument(

In [5]:
#Предполагаемый вектор абзаца для нового документа.
print(model_dm.infer_vector(['привет', 'я', 'пришел'], epochs=100)) 
print(model_dbow.infer_vector(['привет', 'я', 'пришел']))

NameError: ignored

Параметры модели doc2vec

документы (итерируемый из списка TaggedDocument, необязательный) - входной корпус может быть просто списком элементов, но для больших корпораций рассмотрим итеративный, который передает документы непосредственно с диска / сети. Если вы не предоставите документы (или файл corpus_file ), модель останется неинициализированной - используйте, если вы планируете инициализировать ее каким-либо другим способом.

corpus_file ( str , необязательно ) - путь к файлу корпуса в LineSentenceформате. Вы можете использовать этот аргумент вместо документов, чтобы повысить производительность. Необходимо передать только один из документов или аргументы corpus_file (или ни один из них, в этом случае модель не остается инициализированной). Теги документов назначаются автоматически и равны номеру строки, как в TaggedLineDocument.

dm ( {1 , 0} , необязательно ) - определяет алгоритм обучения. Если dm = 1 , используется «распределенная память» (PV-DM). В противном случае используется распределенный пакет слов (PV-DBOW).

vector_size ( int , необязательный ) - размерность векторов объектов.

window ( int , необязательный ) - максимальное расстояние между текущим и прогнозируемым словом в предложении.

альфа (с плавающей точкой , необязательно ) - начальная скорость обучения.

min_alpha ( float , необязательно ) - скорость обучения будет линейно снижаться до min_alpha в процессе обучения.

seed ( int , необязательный ) - Seed для генератора случайных чисел. Начальные векторы для каждого слова засеяны хешем конкатенации слова + str (семя) . Обратите внимание, что для полностью детерминированно-воспроизводимого прогона вы также должны ограничить модель одним рабочим потоком (worker = 1 ), чтобы исключить дрожание порядка в планировании потоков ОС. В Python 3 воспроизводимость между запусками интерпретатора также требует использования переменной среды PYTHONHASHSEED для управления рандомизацией хеша.

min_count ( int , необязательный ) - игнорирует все слова с общей частотой ниже этой.

max_vocab_size ( int , необязательный ) - ограничивает ОЗУ при построении словарного запаса; если есть больше уникальных слов, чем это, то удалите редкие слова. Каждые 10 миллионов типов слов требуют около 1 ГБ оперативной памяти. Установите Нет для отсутствия ограничений.

sample ( float , необязательный ) - порог для настройки того, какие высокочастотные слова случайным образом отбираются, полезный диапазон (0, 1e-5).

working ( int , необязательный ) - используйте эти многочисленные рабочие потоки для обучения модели (= более быстрое обучение на многоядерных машинах).

epochs ( int , необязательный ) - количество итераций (эпох) по всему корпусу.

hs ( {1 , 0} , необязательно ) - если 1, для обучения модели будет использоваться иерархический softmax. Если установлено значение 0, а отрицательное значение не равно нулю, будет использоваться отрицательная выборка.

отрицательный ( int , необязательный ) - если> 0, будет использоваться отрицательная выборка, int для отрицательного указывает, сколько «шумовых слов» следует нарисовать (обычно между 5-20). Если установлено значение 0, отрицательная выборка не используется.

ns_exponent ( float , необязательный ) - показатель степени, используемый для формирования отрицательного распределения выборки. Значение 1,0 выборки точно пропорционально частотам, 0,0 выборки всех слов одинаково, в то время как отрицательное значение выборки низкочастотных слов больше, чем высокочастотных слов. Популярное значение по умолчанию 0,75 было выбрано оригинальной статьей Word2Vec. Совсем недавно, в https://arxiv.org/abs/1804.04212 , Caselles-Dupré, Lesaint и Royo-Letelier предлагают, чтобы другие значения могли работать лучше для рекомендательных приложений.

dm_mean ( {1 , 0} , необязательно ) - если 0, используйте сумму векторов слова контекста. Если 1, используйте среднее. Применяется только когда dm используется в неконкатентивном режиме.

dm_concat ( {1 , 0} , необязательно ) - если 1, использовать конкатенацию векторов контекста, а не сумму / среднее; Обратите внимание, что конкатенация приводит к гораздо большей модели, так как вход больше не является размером одного (сэмплированного или арифметически сложенного) слова-вектора, а размером тега (-ов) и всех слов в контексте, связанных вместе.

dm_tag_count ( int , необязательный ) - ожидаемое постоянное количество тегов документа на документ при использовании режима dm_concat.

dbow_words ( {1 , 0} , необязательно ) - если установлено значение 1, обучает векторы слов (в режиме пропуска граммы) одновременно с обучением doc-vector DBOW; Если 0, обучаются только векторы документов (быстрее).

trim_rule ( функция , необязательно ) -

Правило обрезки словаря, указывает, должны ли определенные слова оставаться в словаре, обрезаться или обрабатываться по умолчанию (отбрасывать, если количество слов <min_count). Может быть None (будет использоваться min_count, смотрите keep_vocab_item()) или вызываться, который принимает параметры (word, count, min_count) и возвращает либо gensim.utils.RULE_DISCARD, gensim.utils.RULE_KEEPлибо gensim.utils.RULE_DEFAULT. Правило, если оно задано, используется только для сокращения словарного запаса во время текущего вызова метода и не сохраняется как часть модели.

Входные параметры имеют следующие типы:
слово (str) - слово, которое мы изучаем

count (int) - частота слова в корпусе

min_count (int) - минимальный порог счета.

обратные вызовы - список обратных вызовов, которые необходимо выполнить / запустить на определенных этапах во время обучения.